In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
seoul = gpd.read_file('LSMD_ADM_SECT_UMD_11.shp', encoding='euc-kr')
seoul

,EMD_CD,EMD_NM,SGG_OID,COL_ADM_SE,GID,geometry
0,11290129,보문동7가,7,11290,5222,"POLYGON ((957864.444 1953662.735, 957878.870 1..."
1,11290103,돈암동,8,11290,5223,"POLYGON ((956352.458 1955412.808, 956340.774 1..."
2,11290139,석관동,9,11290,5224,"POLYGON ((961229.416 1956099.120, 961229.075 1..."
3,11290114,삼선동4가,22,11290,5225,"POLYGON ((957329.114 1954659.435, 957325.653 1..."
4,11290117,동선동2가,29,11290,5232,"POLYGON ((957516.003 1954475.929, 957510.279 1..."
...,...,...,...,...,...,...
462,11230102,용두동,705,11230,2658,"POLYGON ((958007.385 1953418.674, 958018.560 1..."
463,11230107,청량리동,376,11230,2659,"POLYGON ((959280.728 1954765.510, 959294.400 1..."
464,11410112,대현동,1714,11410,2660,"POLYGON ((950576.143 1951255.737, 950585.933 1..."
465,11110157,인의동,348,11110,2661,"POLYGON ((955531.438 1952843.236, 955532.337 1..."


In [3]:
code = pd.read_excel('법정동코드 조회자료.xls')
code

,법정동코드,법정동명
0,1100000000,서울특별시
1,1111000000,서울특별시 종로구
2,1111010100,서울특별시 종로구 청운동
3,1111010200,서울특별시 종로구 신교동
4,1111010300,서울특별시 종로구 궁정동
...,...,...
488,1174010600,서울특별시 강동구 둔촌동
489,1174010700,서울특별시 강동구 암사동
490,1174010800,서울특별시 강동구 성내동
491,1174010900,서울특별시 강동구 천호동


In [4]:
# 자료형 변환
code['법정동코드'] = code['법정동코드'].map(str)

In [10]:
# shp 파일에 법정동 정리
seoul['SI_NAME'] = '모름'
for i in range(len(seoul)):
    k = code[code['법정동코드'].str.contains(seoul['EMD_CD'][i])]
    try: 
       # print(k['법정동명'].str.split(' ').values[0][1])
        seoul.loc[i, 'SI_NAME'] = str(k['법정동명'].str.split(' ').values[0][1])
    except:
        seoul.loc[i, 'SI_NAME'] = '모름'

In [13]:
seoul.to_file("processed_soul.shp", encoding='utf-8')

In [14]:
# 점수화를 위한 데이터 읽기
data = pd.read_csv('Calculated_B_pop.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)

In [19]:
data['안전지수'] = (data['경찰서'] + data['소방서']) / data['법정동 인구밀도']
data['편의지수'] = (data['백화점'] + data['편의점']) / data['법정동 인구밀도']
data['교통지수'] = (data['버스'] + data['지하철']) / data['법정동 인구밀도']
data['의료지수'] = (data['병원'] + data['약국']) / data['법정동 인구밀도']
data['문화지수'] = (data['공원'] + data['영화관']) / data['법정동 인구밀도']

data = data[['B_시군구', 'B_동', '안전지수', '편의지수', '교통지수', '의료지수', '문화지수']]

data.columns = ['B_시군구', 'B_동', '안전지수', '편의지수', '교통지수', '의료지수', '문화지수']
data

,B_시군구,B_동,안전지수,편의지수,교통지수,의료지수,문화지수
0,강남구,개포동,0.000135,0.001244,0.002724,0.003195,0.000000
1,강남구,논현동,0.000088,0.003564,0.001443,0.013843,0.000088
2,강남구,대치동,0.000024,0.001220,0.000825,0.003481,0.000000
3,강남구,도곡동,0.000018,0.000689,0.000870,0.002447,0.000000
4,강남구,삼성동,0.000159,0.002762,0.001778,0.005873,0.000063
...,...,...,...,...,...,...,...
455,중랑구,면목동,0.000033,0.000576,0.000824,0.001328,0.000013
456,중랑구,묵동,0.000034,0.000576,0.000881,0.001017,0.000017
457,중랑구,상봉동,0.000000,0.000640,0.000606,0.001212,0.000034
458,중랑구,신내동,0.000104,0.000598,0.001820,0.001768,0.000026


In [25]:
data['key'] = data['B_시군구'] + ' ' + data['B_동']
seoul['key'] = seoul['SI_NAME'] + ' ' + seoul['EMD_NM']

/var/folders/2d/k1h9bjt9385d884qrqvzk1880000gn/T/ipykernel_38614/4284616142.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['key'] = data['B_시군구'] + ' ' + data['B_동']


In [28]:
# 데이터 합치기
all_data = pd.merge(data, seoul, left_on='key', right_on='key', how='outer')
all_data

,B_시군구,B_동,안전지수,편의지수,교통지수,의료지수,문화지수,key,EMD_CD,EMD_NM,SGG_OID,COL_ADM_SE,GID,geometry,SI_NAME
0,강남구,개포동,0.000135,0.001244,0.002724,0.003195,0.000000,강남구 개포동,11680103,개포동,379,11680,2773,"POLYGON ((960785.917 1943241.515, 960834.210 1...",강남구
1,강남구,논현동,0.000088,0.003564,0.001443,0.013843,0.000088,강남구 논현동,11680108,논현동,372,11680,3011,"POLYGON ((959213.864 1947175.811, 959269.830 1...",강남구
2,강남구,대치동,0.000024,0.001220,0.000825,0.003481,0.000000,강남구 대치동,11680106,대치동,373,11680,3068,"POLYGON ((961835.767 1945742.818, 961897.395 1...",강남구
3,강남구,도곡동,0.000018,0.000689,0.000870,0.002447,0.000000,강남구 도곡동,11680118,도곡동,376,11680,2622,"POLYGON ((960475.467 1944210.535, 960490.428 1...",강남구
4,강남구,삼성동,0.000159,0.002762,0.001778,0.005873,0.000063,강남구 삼성동,11680105,삼성동,371,11680,2709,"POLYGON ((961604.182 1947398.279, 961691.696 1...",강남구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,용산구 주성동,11170134,주성동,160,11170,2794,"POLYGON ((955589.571 1947397.685, 955606.203 1...",용산구
463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,중구 의주로2가,11140172,의주로2가,329,11140,2699,"POLYGON ((953235.328 1951376.493, 953235.408 1...",중구
464,NaN,NaN,NaN,NaN,NaN,NaN,NaN,중구 남학동,11140140,남학동,318,11140,2702,"POLYGON ((955041.183 1951319.438, 955034.516 1...",중구
465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,중구 주자동,11140141,주자동,302,11140,2650,"POLYGON ((954880.685 1951441.596, 954881.884 1...",중구


In [29]:
all_data.shape

(467, 15)

In [30]:
all_data.isnull().sum()

B_시군구         7
B_동           7
안전지수          7
편의지수          7
교통지수          7
의료지수          7
문화지수          7
key           0
EMD_CD        0
EMD_NM        0
SGG_OID       0
COL_ADM_SE    0
GID           0
geometry      0
SI_NAME       0
dtype: int64

# 없는 동 있다!!!

In [31]:
all_data[all_data.B_시군구.isnull()]

,B_시군구,B_동,안전지수,편의지수,교통지수,의료지수,문화지수,key,EMD_CD,EMD_NM,SGG_OID,COL_ADM_SE,GID,geometry,SI_NAME
460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,종로구 사간동,11110144,사간동,329,11110,2908,"POLYGON ((954215.199 1953337.567, 954217.663 1...",종로구
461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,종로구 봉익동,11110152,봉익동,352,11110,2843,"POLYGON ((955189.345 1952840.786, 955192.385 1...",종로구
462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,용산구 주성동,11170134,주성동,160,11170,2794,"POLYGON ((955589.571 1947397.685, 955606.203 1...",용산구
463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,중구 의주로2가,11140172,의주로2가,329,11140,2699,"POLYGON ((953235.328 1951376.493, 953235.408 1...",중구
464,NaN,NaN,NaN,NaN,NaN,NaN,NaN,중구 남학동,11140140,남학동,318,11140,2702,"POLYGON ((955041.183 1951319.438, 955034.516 1...",중구
465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,중구 주자동,11140141,주자동,302,11140,2650,"POLYGON ((954880.685 1951441.596, 954881.884 1...",중구
466,NaN,NaN,NaN,NaN,NaN,NaN,NaN,종로구 훈정동,11110150,훈정동,332,11110,2662,"POLYGON ((955466.246 1953099.971, 955471.458 1...",종로구


In [32]:
all_data = pd.merge(data, seoul, left_on='key', right_on='key', how='inner')
all_data.drop('key', axis=1, inplace=True)
all_data.drop('B_시군구', axis=1, inplace=True)
all_data.drop('B_동', axis=1, inplace=True)

In [35]:
all_data.columns = ['Safety', 'Conv', 'Traffic', 'Medical', 'Culture', 'EMD_CD', 'EMD_NM', 'SGG_OID',
       'COL_ADM_SE', 'GID', 'geometry', 'SI_NAME']

In [36]:
all_data[['Safety', 'Conv', 'Traffic', 'Medical', 'Culture', 'EMD_CD']].to_csv('BaB_cnt_cal_pop.csv')


In [37]:
all_data

,Safety,Conv,Traffic,Medical,Culture,EMD_CD,EMD_NM,SGG_OID,COL_ADM_SE,GID,geometry,SI_NAME
0,0.000135,0.001244,0.002724,0.003195,0.000000,11680103,개포동,379,11680,2773,"POLYGON ((960785.917 1943241.515, 960834.210 1...",강남구
1,0.000088,0.003564,0.001443,0.013843,0.000088,11680108,논현동,372,11680,3011,"POLYGON ((959213.864 1947175.811, 959269.830 1...",강남구
2,0.000024,0.001220,0.000825,0.003481,0.000000,11680106,대치동,373,11680,3068,"POLYGON ((961835.767 1945742.818, 961897.395 1...",강남구
3,0.000018,0.000689,0.000870,0.002447,0.000000,11680118,도곡동,376,11680,2622,"POLYGON ((960475.467 1944210.535, 960490.428 1...",강남구
4,0.000159,0.002762,0.001778,0.005873,0.000063,11680105,삼성동,371,11680,2709,"POLYGON ((961604.182 1947398.279, 961691.696 1...",강남구
...,...,...,...,...,...,...,...,...,...,...,...,...
455,0.000033,0.000576,0.000824,0.001328,0.000013,11260101,면목동,33,11260,2945,"POLYGON ((965581.683 1954297.513, 965569.711 1...",중랑구
456,0.000034,0.000576,0.000881,0.001017,0.000017,11260104,묵동,34,11260,2646,"POLYGON ((963563.609 1956623.336, 963458.669 1...",중랑구
457,0.000000,0.000640,0.000606,0.001212,0.000034,11260102,상봉동,35,11260,2645,"POLYGON ((962085.814 1954886.570, 962108.994 1...",중랑구
458,0.000104,0.000598,0.001820,0.001768,0.000026,11260106,신내동,49,11260,2924,"POLYGON ((963354.850 1956998.403, 963387.913 1...",중랑구
